<a href="https://colab.research.google.com/github/Gopib03/LLM-Pretrained/blob/main/Fine_tune_LLM_using_trl_and_the_SFTTrainer_with_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 32.3 MB/s eta 0:00:00


In [2]:
!pip install -q trl peft datasets transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 42.0 MB/s eta 0:00:00


PEFT methods can be combined with TRL for fine-tuning to reduce memory requirements. We can pass the LoraConfig to the model when loading it.

In [1]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

In [6]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType


device = "cuda" if torch.cuda.is_available() else "cpu"

dataset = load_dataset("HuggingFaceTB/smoltalk", "all")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['messages', 'source'],
        num_rows: 1043917
    })
    test: Dataset({
        features: ['messages', 'source'],
        num_rows: 54948
    })
})


In [8]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
MAX_LEN = 16384

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # REQUIRED

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="auto"
)

In [9]:
##Setup chat template

tokenizer.chat_template = """
{% for message in messages %}
{% if message['role'] == 'user' %}
User: {{ message['content'] }}
{% elif message['role'] == 'assistant' %}
Assistant: {{ message['content'] }}
{% elif message['role'] == 'system' %}
System: {{ message['content'] }}
{% endif %}
{% endfor %}
"""

In [11]:
##LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

# Apply LoRA to model
model = get_peft_model(model, peft_config)


In [14]:
# Chat formatting + tokenization

def format_and_tokenize(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False
    )

    return tokenizer(
        text,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
    )

dataset = dataset.map(
    format_and_tokenize,
    batched=True,
    batch_size=16,
    num_proc=16,
    remove_columns=dataset["train"].column_names,
)


Map (num_proc=16):   0%|          | 0/1043917 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/54948 [00:00<?, ? examples/s]

In [16]:
## Data collator

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [21]:
# Training Config
training_args = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,
    per_device_train_batch_size=1,  # safe for 16k context
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
    fp16=True,
    gradient_checkpointing=True,
    save_total_limit=3,
)

In [22]:
# Initialize SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
)

Truncating train dataset:   0%|          | 0/1043917 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/54948 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [24]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gopib3456 (gopib3456-na) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,1.460300,1.508940,1.499023,127171.000000,0.644501
100,1.658100,1.444750,1.441558,258119.000000,0.655444
150,1.268600,1.393830,1.346131,370561.000000,0.666017
200,1.425100,1.363020,1.359697,490326.000000,0.673487
250,1.368200,1.352420,1.328618,613389.000000,0.674455
300,1.315400,1.344918,1.363988,734614.000000,0.676273
350,1.326200,1.339125,1.310741,854907.000000,0.677325
400,1.223600,1.333805,1.336952,974546.000000,0.678153
450,1.331100,1.329709,1.320980,1101651.000000,0.678988
500,1.283200,1.325709,1.326205,1232350.000000,0.680112


TrainOutput(global_step=1000, training_loss=1.3421095628738404, metrics={'train_runtime': 47809.0683, 'train_samples_per_second': 0.084, 'train_steps_per_second': 0.021, 'total_flos': 2.330511853698048e+16, 'train_loss': 1.3421095628738404, 'epoch': 0.0038317222537807122})